<a href="https://colab.research.google.com/github/ShayHXY1017/Python-for-Public-Policy-Final-Project/blob/main/EU_ETS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Background Intro

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [13]:
raw = pd.read_csv('/Heat & Transportation Emission Data.csv', encoding='utf-16', sep='\t')
print(raw.head())
print(raw.columns)

                                        Sector Name  Gas  Country  \
0  1.A.1.a - Public Electricity and Heat Production  CO₂  Austria   
1  1.A.1.a - Public Electricity and Heat Production  CO₂  Austria   
2  1.A.1.a - Public Electricity and Heat Production  CO₂  Austria   
3  1.A.1.a - Public Electricity and Heat Production  CO₂  Austria   
4  1.A.1.a - Public Electricity and Heat Production  CO₂  Austria   

   Year of Date           t  
0          1990  11,056,340  
1          1991  11,767,755  
2          1992   8,599,200  
3          1993   8,430,924  
4          1994   8,712,226  
Index(['Sector Name', 'Gas', 'Country', 'Year of Date', 't'], dtype='object')
